# Joint Representation Learning

In many network problems, our network might be more than just its topology (its collection of nodes and edges). If we were investigating a social network, we might have access to extra information about each person -- their gender, for instance, or their age. If we were investigating a brain network, we might have information about the physical location of neurons, or the volume of a brain region. When we we embed a network, it seems like we should be able to use these extra bits of information - called the "features" or "covariates" of a network - to somehow improve our analysis. The techniques and tools that we'll explore in this section use both the covariates and the topology of a network to create and learn from new representations of our network. Because they jointly use both the topology of our network and its extra covariate information, these techniques and tools are called joint representation learning.

There are two primary reasons that we might want to explore using node covariates in addition to topological structure. First, they might improve our standard embedding algorithms, like Laplacian and Adjacency Spectral Embedding. For example, if the latent structure of the covariates of a network lines up with the latent structure of its topology, then we might be able to reduce noise when we embed, even if the communities in our network don't overlap perfectly with the communities in our covariates. Second, figuring out what the clusters of an embedding actually mean can sometimes be difficult and covariates create a natural structure in our network that we can explore. Covariate information in brain networks telling us where in the brain each node is, for instance, might let us better understand the types of characteristics that distinguish between different brain regions.

In this section, we'll explore different ways to learn from our data when we have access to these covariates of a network in addition to its topological structure. We'll explore *Covariate-Assisted Spectral Embedding* (CASE), a variation on Spectral Embedding. In CASE, instead of embedding just the adjacency matrix or one of the many versions of its Laplacian, we'll combine the Laplacian and our covariates into a new matrix and embed that.

A good way to illustrate how using covariates might help us is to use a model in which some of our community information is in the covariates and some is in our topology. Using the Stochastic Block Model, we’ll create a simulation using three communities: the first and second community will be indistinguishable in the topological structure of a network, and the second and third community will be indistinguishable in its covariates. By combining the topology and the covariates, we'll get a nice embedding that lets us find three distinct community clusters.

### Stochastic Block Model

Suppose we have a network generated from a Stochastic Block Model (or, commonly, SBM) with three communities (we promise). There are 1500 nodes, with 500 in each community. Because the $3 \times 3$ block probability matrix that generated this SBM has the same probability values (.3) in its upper-left $2 \times 2$ square, the first two communities are completely indistinguishable: Any random node in the first community will have exactly the same 15% chance of being connected to another node in the first community and to a node in the second community. As a result, in our adjacency matrix, we'll see the nodes in communities one and two as a single giant block. The third community will be distinct, with only a small probability (.15) that nodes in it will connect to nodes in either of the other two communities. The topological structure shown by the adjacency matrix can't separate three groups by itself.

In [ ]:
import warnings
warnings.filterwarnings("ignore")  # TODO: don't do this, fix scatterplot

import numpy as np
np.random.seed(42)

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from graspologic.simulations import sbm
from graspologic.plot import heatmap

# Start with some simple parameters
N = 1500  # Total number of nodes
n = N // 3  # Nodes per community
p, q = .3, .15
B = np.array([[.3, .3, .15],
              [.3, .3, .15],
              [.15, .15, .3]])  # Our block probability matrix

# Make and visualize our Stochastic Block Model
A, labels = sbm([n, n, n], B, return_labels = True)

# make the colorbar look nice
fig, ax = plt.subplots(figsize=(10,10))

def plot_heatmap(A, ax, title="A Stochastic Block Model", show_cbar=True):
    cmap = matplotlib.colors.ListedColormap(["white", 'black'])
    with sns.plotting_context("talk", font_scale=1):
        ax = sns.heatmap(A, cmap=cmap, ax=ax, square=True,
                         cbar_kws=dict(shrink=0.7), xticklabels=False,
                         yticklabels=False)
        ax.set_title(title)
        cbar = ax.collections[0].colorbar
        if show_cbar:
            cbar.set_ticks([0.25, .75])
            cbar.set_ticklabels(['No Edge', 'Edge'])
            cbar.ax.set_frame_on(True)
        else:
            cbar.remove()
            
        
plot_heatmap(A, ax)

If we wanted to embed this graph using our Laplacian or Adjacency Spectral Embedding methods, we'd find the first and second communities layered on top of each other (though we wouldn't be able to figure that out from our embedding if we didn't cheat by knowing in advance which community each node is supposed to belong to). The python code below embeds our latent positions down to two dimensions with a Laplacian Spectral Embedding, and then plots the results, color-coding each node by its true community.

In [ ]:
from graspologic.embed import LaplacianSpectralEmbed as LSE
from graspologic.utils import to_laplacian
import matplotlib.pyplot as plt
from sklearn.utils.extmath import randomized_svd
import seaborn as sns
from graspologic.plot import pairplot

def embed(A, *, dimension):
    latents, _, _ = randomized_svd(A, n_components=dimension)
    return latents

def plot_latents(latent_positions, *, title, labels, ax=None):
    if ax is None:
        ax = plt.gca()
    plot = sns.scatterplot(latent_positions[:, 0], latent_positions[:, 1], hue=labels, 
                           linewidth=0, s=10, ax=ax, palette="Set1")
    plot.set_title(title, wrap=True);
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.legend(loc="upper right", title="Community")
    
    return plot

L = to_laplacian(A, form="R-DAD")
L_latents = embed(L, dimension=2)
plot = plot_latents(L_latents, title="Latent positions when we\n only embed the Laplacian", 
             labels=labels)

We'd like to use extra information to more clearly distinguish between the first and second community. We don't have this information in our network: it needs to come from somewhere else.

### Covariates

But we're in luck - we have extra in a set of covariates for each node! These covariates contain the extra information we need that allows us to separate our first and second community. However, with only these extra covariate features, we can no longer distinguish between the last two communities - they contain the same information.

Below is a visualization of our covariates. Each node is associated with its own group of 30 covariates (thirty being chosen primarily to visualize what's happening more clearly). We'll organize this information into a matrix, where the $i_{th}$ row contains the covariates associated with node $i$. Remember that we have 1500 nodes in our network, so there will be 1500 rows. We'll draw all the covariates for each node from the same Beta distribution (with values ranging from 0 to 1), but the nodes in the first community will be drawn from a different Beta distribution than the nodes in the last two. As a result, the first community is represented by the lighter-colored rows, and the last two are represented by the darker-colored rows.

In [ ]:
import numpy as np
from scipy.stats import bernoulli, beta
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

def gen_covariates(N=1500):
    make_community = lambda a, b: beta.rvs(a, b, size=(N//3, 30))
    c1 = make_community(2, 5)
    c2 = make_community(2, 2)
    c3 = make_community(2, 2)

    covariates = np.vstack((c1, c2, c3))
    return covariates
    

# Generate a covariate matrix
X = gen_covariates(N=N)

# Plot and make the axis look nice
fig, ax = plt.subplots(figsize=(5, 8))
ax = sns.heatmap(X, ax=ax, cmap="rocket_r")
ax.set(title="Visualization of the covariates", xticks=[], 
       ylabel="Nodes (each row is a node)",
       xlabel="Covariates for each node (each column is a covariate)");

We can play almost the same game here as we did with the Laplacian. If we embed the information contained in this matrix of covariates into lower dimensions, we can see the reverse situation as before - the first community is separate, but the last two are overlayed on top of each other.

In [ ]:
XXt = X@X.T
X_latents = embed(XXt, dimension=2)
plot_latents(X_latents, title="Latent positions when we\n only embed our covariates", 
             labels=labels);

We want full separation between all three communities, so we need some kind of representation of our network that allows us to use both the information in the topology and the information in the covariates. This is where CASE comes in.

## Covariate-Assisted Spectral Embedding

<i>Covariate-Assisted Spectral Embedding</i>, or CASE<sup>1</sup>, is a simple way of combining our network and our covariates into a single model. In the most straightforward version of CASE, we combine the network's regularized Laplacian matrix $L$ and a function of our covariate matrix $XX^\top$. Here, $X$ is just our covariate matrix, in which row $i$ contains the covariates associated with node $i$. Notice the word "regularized" - This means (from the Laplacian section earlier) that our Laplacian looks like $L = L_{\tau} = D_{\tau}^{-1/2} A D_{\tau}^{-1/2}$.

```{note}
Suppose that $X$ only contains 0's and 1's. To interpret $XX^T$, notice from linear algebra that we're effectively taking the weighted sum - or, in math parlance, the dot product - of each row of $X$ with each other row, because the transpose operation turns rows into columns. Now, look at what happens below when we take the dot product of two vectors with only 0's and 1's in them:

\begin{align}
\begin{bmatrix}
1 & 1 & 1
\end{bmatrix} \cdot 
\begin{bmatrix}
0 \\
1 \\
1 \\
\end{bmatrix} = 1\times 0 + 1\times 1 + 1\times 1 = 2
\end{align}

If there are two overlapping 1's in the same position of the left vector and the right vector, then there will be an additional 1 added to their weighted sum. So, in the case of the binary $XX^T$, when we matrix-multiply a row of $X$ by a column of $X^T$, the resulting value, $(XX^T)_{i, j}$, will be equal to the number of shared locations in which vectors $i$ and $j$ both have ones.
```

A particular value in $XX^\top$, $(XX^\top)_{i, j}$, can be interpreted as measuring the "agreement" or "similarity" between row $i$ and row $j$ of our covariate matrix. The higher the value, the more the two rows share 1's in the same column. The result is a matrix that looks fairly similar to our Laplacian!  

The following Python code generates both our SBM and our covariate similarity matrix $XX^\top$. We'll also normalize the rows of our covariate matrix to have unit length using scikit-learn - this is because we want the scale for our covariate matrix to be roughly the same as the scale for our adjacency matrix. Later, we'll use a tuning coefficient to help with this as well.

In [ ]:
import numpy as np
from graspologic.utils import to_laplacian
from graspologic.simulations import sbm
from sklearn.preprocessing import normalize

def gen_sbm(p=.3, q=.15, N=1500):
    """
    Generate an adjacency matrix.
    """
    n = N // 3
    B = np.full((3, 3), q)
    B[np.diag_indices_from(B)] = p
    A = sbm([n, n, n], B, return_labels=True)

    return A
    
def gen_covariates(N=1500):
    make_community = lambda a, b: beta.rvs(a, b, size=(N//3, 30))
    c1 = make_community(2, 5)
    c2 = make_community(2, 2)
    c3 = make_community(2, 2)

    covariates = np.vstack((c1, c2, c3))
    return covariates

# Generate a covariate matrix
X = gen_covariates(N=N)
X = normalize(X, axis=0)

L = to_laplacian(A, form="R-DAD")
XXt = X@X.T

You can see what our two matrices look like below. As you can see, each matrix contains information about our communities that the other doesn't have.

In [ ]:
# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5), constrained_layout=True)
L_ax = plot_heatmap(L, title=r"Regularized Laplacian", ax=axs[0])
X_ax = heatmap(XXt, title="Covariate matrix times \nits transpose", ax=axs[1]);

CASE is simply a weighted sum of these two matrices. The weight (here called $\alpha$) is multiplied by $XX^\top$ - that way, both matrices contribute an equal amount of useful information to the embedding. 

$$
L + aXX^\top
$$

### Exploring Possible Weights

An obvious question here is how to weight the covariates. If we simply summed the two matrices by themselves, we'd unfortunately be in a situation where whichever matrix contained larger numbers would dominate over the other. In our current setup, without a weight on $XX^\top$, the covariates of our network would dominate over its topology.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
heatmap(L + X@X.T, title="Summing the two matrices \nwithout a weight", ax=axs[0], context='notebook');
plot_latents(embed(L + X@X.T, dimension=2), title="Embedding the summed \nmatrix without a weight", 
             ax=axs[1], labels=labels);

What do different potential weights look like? Let's do a comparison. Below you can see 9 possible weights on $XX^\top$, ranging between $10^{-5}$ and 100.

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20, 20))
for a, ax in zip(np.geomspace(10e-5, 100, num=9), axs.flat):
    l_ = embed(L+a*XXt, dimension=2)
    plot_latents(l_, title=f"weight: {a:.3f}", labels=labels, ax=ax)
    ax.get_legend().remove()

fig.suptitle(r"Comparison of embeddings for different weights on $XX^\top$", y=1, fontsize=25);
plt.tight_layout()

It looks like we'd probably want a weight somewhere between 0.01 and 0.5  - then, we'll have three communities which are fairly distinct from each other, implying that we're pulling good information from both our network's topology and its covariates. We could just pick our weight manually, but it would be nice to have some kind of algorithm or equation which lets us pick a reasonable value automatically.

### Finding a Reasonable Weight Automatically

In general, we'd like to embed in a way that lets us distinguish between communities. This means that if we knew which community each node belonged to, we'd like to be able to correctly retrieve the correct commmunities for each node as possible with a clustering algorithm after embedding. This also implies that the communities will be as distinct as possible.

We already found a range of possible weights, embedded our combined matrix for every value in this range, and then looked at which values produced the best clustering. But, how do we find a weight which lets us consistently use useful information from both the topology and the covariates?

When we embed symmetric matrices, keep in mind that the actual points we're plotting are the components of the eigenvectors with the biggest eigenvalues. When we embed into two-dimensional space, for instance, the X-axis values of our points are the components of the eigenvector with the biggest eigenvalue, and the Y-axis values are the components of the eigenvector with the second-biggest eigenvalue. This means that we should probably be thinking about how much information the Laplacian and $XX^\top$ contributes to the biggest eigenvalue/eigenvector pairs. 

Thinking about this more, if we have a small weight, $XX^\top$ will contribute only a small amount to the biggest eigenvalue/vector pair. If we have a large weight, $XX^\top$ will contribute a large amount to the biggest eigenvalue/vector pair. The weight that causes the Laplacian and $XX^\top$ to contribute the same amount of information, then, is just the ratio of the biggest eigenvalue of $L$ and the biggest eigenvalue of $XX^\top$:

$$
weight = \frac{\lambda_1 (L)}{\lambda_1 (XX^\top)}
$$

Let's check what happens when we combine our Laplacian and covariates matrix using the weight described in the equation above. Our embedding works the same as it does in Laplacian Spectral Embedding: we decompose our combined matrix using Singular Value Decomposition, truncating the columns, and then we visualize the rows of the result. Remember, we'll be embedding $L + \alpha XX^\top$. We'll embed all the way down to two dimensions, just to make visualization simpler. As you can see, the combined matrix has good separation between all three communities.

In [ ]:
from scipy.sparse.linalg import eigsh
leading_eigval_L, = eigsh(L, return_eigenvectors=False, k=1)
leading_eigval_XXt, = eigsh(XXt, return_eigenvectors=False, k=1)

weight = leading_eigval_L / leading_eigval_XXt

L_ = L + weight*XXt
latents = embed(L_, dimension=2)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
heatmap(L_, title=f"Our Combined Laplacian and \ncovariates matrix with a weight of {weight:.2f}", ax=axs[0])
plot_latents(latents, 
             title=f"Embedding of the combined matrix", 
             labels=labels, ax=axs[1]);

plt.tight_layout()

Success! Just to see how big the difference is, below we can see (from left to right) a comparison of our network's latent position when we only use its topological information, when we only use the information contained in its covariates, and finally our embedding using the weight we found.

In [ ]:
from graspologic.embed import LaplacianSpectralEmbed as LSE


# Plot
fig, axs = plt.subplots(1, 3, figsize=(15,5))
plot_latents(L_latents, title="Latent positions when we only use the Laplacian", 
             labels=labels, ax=axs[0])
plot_latents(X_latents, title="Latent positions when we only use our covariates", 
             labels=labels, ax=axs[1]);
plot_latents(latents_, title="Latent positions when we combine\n our network and its covariates", 
             labels=labels, ax=axs[2])

plt.tight_layout()

## Variations on CASE

We don't always want to embed $L + \alpha XX^\top$. Using the regularized Laplacian by itself, for instance, isn't always best. Here are some situations where we might consider using a different equation to combine the topological and the covariate information in our network.

*non-assortative*  
If your graph is *non-assortative* - meaning, the between-block probabilities are greater than the within-block probabilities - it's better to square our Laplacian. This gets rid of a lot of annoying negative eigenvalues, and we end up embedding $LL + aXX^\top$.  

*big graphs*  
In large graphs, we'd like to perform as few operations on our two matrices as possible prior to embedding (since any computation we do on huge matrices is pretty expensive and can take awhile). Here, we can use a variant on classical correlation analysis<sup>4</sup> and simply embed $LX$.

## Using Graspologic

Graspologic's CovariateAssistedSpectralEmbedding class implements CASE directly. The following code applies CASE to reduce the dimensionality of $L + aXX^T$ down to two dimensions, and then plots the latent positions to show the clustering. You can also try the above variations on CASE with the `embedding_alg` parameter.

In [ ]:
import graspologic

casc = graspologic.embed.CovariateAssistedEmbedding(embedding_alg="assortative", n_components=2, tuning_runs=10)
latents = casc.fit_transform(A, covariates=X)
plot_latents(latents, title="Embedding our model using graspologic", labels=labels);

#### References

[1] N. Binkiewicz, J. T. Vogelstein, K. Rohe, Covariate-assisted spectral clustering, Biometrika, Volume 104, Issue 2, June 2017, Pages 361–377, https://doi.org/10.1093/biomet/asx008  
[2] Lloyd, S. (1982). Least squares quantization in PCM. IEEE transactions on information theory, 28(2), 129-137.  
[3] https://scikit-learn.org/stable/modules/clustering.html#k-means
[4] Hotelling, H. (1936). Relations between two sets of variates. Biometrika 28, 321–77.  